<a href="https://colab.research.google.com/github/swarupa29/twitter_event_detection/blob/master/element_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [ ]:
df=pd.read_csv(r'../euro2012_V2_SV_LabeledDataset.csv')

In [ ]:
x=df['processed_string']
len(x)

190

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from keras.utils import to_categorical

In [ ]:
onehot_repr=[one_hot(words,5000)for words in x] 
print(onehot_repr[4])

[48, 4864, 3191, 770, 2995, 3638, 4363, 511, 2392, 4747, 2971, 4800, 4864, 770, 2995, 4402, 4363, 2684, 1793, 121, 4147]


In [ ]:
y=df['labeled_string']
len(y)

190

In [ ]:
tag_to_ix={}
for sentence in y:
    for tag in sentence.split():
            if tag not in tag_to_ix:
                tag_to_ix[tag] = len(tag_to_ix)
print(tag_to_ix)

{'O': 0, 'S-PAR': 1, 'S-TRIG': 2, 'S-DT': 3, 'B-PAR': 4, 'E-PAR': 5, 'B-DT': 6, 'I-DT': 7, 'E-DT': 8, 'S-LOC': 9, 'S-VID': 10, 'I-PAR': 11, 'B-TRIG': 12, 'E-TRIG': 13}


In [ ]:
tags = []
for tag in (df["labeled_string"].values):
        tags.append(tag)
print(len(tags))

190


In [ ]:
y_final=[]
for i in tags:
    tg=[tag_to_ix[j] for j in i.split()]
    y_final.append(tg)
print(y_final[40])      
        


[0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 0]


In [ ]:
from keras.preprocessing.sequence import pad_sequences
y_FIN = pad_sequences(maxlen=30, sequences=y_final, padding="post", value=tag_to_ix["O"])
y_FINAL=[to_categorical(i, num_classes=18) for i in y_FIN]
print(len(y_FINAL))

190


In [ ]:
x_FINAL=pad_sequences(onehot_repr,padding='post',maxlen=30)
print(len(x_FINAL))

190


In [ ]:
from keras_contrib.layers import *
from keras.losses import *
import keras.metrics
import tensorflow as tf
import keras as k

In [ ]:
#building the model
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional,Flatten
from keras_contrib.layers import CRF
#from tf_crf_layer.loss import crf_loss, ConditionalRandomFieldLoss

In [ ]:
MAX_LEN=30
adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)
input = Input(shape=(MAX_LEN,))
model = Embedding(input_dim=5000, output_dim=300, # n_words + 2 (PAD & UNK)
                  input_length=30)(input) 
model = Bidirectional(LSTM(units=100, return_sequences=True,
                           recurrent_dropout=0.1))(model)  
model = TimeDistributed(Dense(1, activation="sigmoid"))(model) 
#model = TimeDistributed(Dense(1, activation="tanh"))(model)
crf = CRF(18) 
out = crf(model)  # output
model = Model(input, out)
model.compile(loss=crf.loss_function, optimizer=adam, metrics=[crf.accuracy])
model.summary()

C:\conda\Anaconda3\lib\site-packages\keras_contrib-2.0.8-py3.7.egg\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
C:\conda\Anaconda3\lib\site-packages\keras_contrib-2.0.8-py3.7.egg\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 30)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 30, 300)           1500000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 30, 200)           320800    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 1)             201       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 1)             2         
_________________________________________________________________
crf_2 (CRF)                  (None, 30, 18)            396       
Total params: 1,821,399
Trainable params: 1,821,399
Non-trainable params: 0
_________________________________________________

In [ ]:
import numpy as np
X_final=np.array(x_FINAL)
y_final=np.array(y_FINAL)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=20)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=110,batch_size=64)

C:\conda\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 127 samples, validate on 63 samples
Epoch 1/110
127/127 [==============================] - 3s 21ms/step - loss: 2.9229 - crf_viterbi_accuracy: 7.8538e-04 - val_loss: 2.9161 - val_crf_viterbi_accuracy: 5.2910e-04
Epoch 2/110
127/127 [==============================] - 0s 3ms/step - loss: 2.9123 - crf_viterbi_accuracy: 7.8952e-04 - val_loss: 2.9057 - val_crf_viterbi_accuracy: 5.2910e-04
Epoch 3/110
127/127 [==============================] - 0s 3ms/step - loss: 2.9019 - crf_viterbi_accuracy: 7.8125e-04 - val_loss: 2.8956 - val_crf_viterbi_accuracy: 5.2910e-04
Epoch 4/110
127/127 [==============================] - 0s 3ms/step - loss: 2.8919 - crf_viterbi_accuracy: 7.8538e-04 - val_loss: 2.8861 - val_crf_viterbi_accuracy: 5.2910e-04
Epoch 5/110
127/127 [==============================] - 0s 3ms/step - loss: 2.8827 - crf_viterbi_accuracy: 7.8952e-04 - val_loss: 2.8774 - val_crf_viterbi_accuracy: 5.2910e-04
Epoch 6/110
127/127 [==============================] - 0s 3ms/step - loss: 2.87

127/127 [==============================] - 0s 3ms/step - loss: 2.4193 - crf_viterbi_accuracy: 0.3297 - val_loss: 2.4131 - val_crf_viterbi_accuracy: 0.3296
Epoch 49/110
127/127 [==============================] - 0s 3ms/step - loss: 2.4079 - crf_viterbi_accuracy: 0.3296 - val_loss: 2.4017 - val_crf_viterbi_accuracy: 0.3296
Epoch 50/110
127/127 [==============================] - 0s 3ms/step - loss: 2.3964 - crf_viterbi_accuracy: 0.3296 - val_loss: 2.3903 - val_crf_viterbi_accuracy: 0.3296
Epoch 51/110
127/127 [==============================] - 0s 3ms/step - loss: 2.3850 - crf_viterbi_accuracy: 0.3297 - val_loss: 2.3790 - val_crf_viterbi_accuracy: 0.3296
Epoch 52/110
127/127 [==============================] - 0s 3ms/step - loss: 2.3736 - crf_viterbi_accuracy: 0.3297 - val_loss: 2.3676 - val_crf_viterbi_accuracy: 0.3296
Epoch 53/110
127/127 [==============================] - 0s 3ms/step - loss: 2.3622 - crf_viterbi_accuracy: 0.3297 - val_loss: 2.3563 - val_crf_viterbi_accuracy: 0.3296
Epoch

Epoch 97/110
127/127 [==============================] - 0s 3ms/step - loss: 1.8749 - crf_viterbi_accuracy: 0.9680 - val_loss: 1.8705 - val_crf_viterbi_accuracy: 0.9651
Epoch 98/110
127/127 [==============================] - 0s 3ms/step - loss: 1.8641 - crf_viterbi_accuracy: 0.9680 - val_loss: 1.8597 - val_crf_viterbi_accuracy: 0.9651
Epoch 99/110
127/127 [==============================] - 0s 3ms/step - loss: 1.8533 - crf_viterbi_accuracy: 0.9680 - val_loss: 1.8490 - val_crf_viterbi_accuracy: 0.9651
Epoch 100/110
127/127 [==============================] - 0s 3ms/step - loss: 1.8426 - crf_viterbi_accuracy: 0.9680 - val_loss: 1.8383 - val_crf_viterbi_accuracy: 0.9651
Epoch 101/110
127/127 [==============================] - 0s 3ms/step - loss: 1.8320 - crf_viterbi_accuracy: 0.9679 - val_loss: 1.8277 - val_crf_viterbi_accuracy: 0.9651
Epoch 102/110
127/127 [==============================] - 0s 3ms/step - loss: 1.8212 - crf_viterbi_accuracy: 0.9680 - val_loss: 1.8170 - val_crf_viterbi_accura

In [ ]:
y_pred=model.predict(X_test)
print(y_test)

[[[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 ...

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


In [ ]:
y_pred1 = np.argmax(y_pred, axis=-1)
print(y_pred1[9])
y_test1=np.argmax(y_test,axis=-1)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
for i in range(len(X_test)):
    print("test case ",i)
    print(y_test1[i])
    print(y_pred1[i])

test case  0
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
test case  1
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
test case  2
[1 2 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
test case  3
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
test case  4
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
test case  5
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
test case  6
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
test case  7
[0 3 0 2 0 0 0 0 0 0 0 0 0 0

In [ ]:
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=y_pred, y_true=y_test)
print(report)

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1824
           1       0.50      0.11      0.18        27
           2       0.00      0.00      0.00        17
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         8
           5       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00    

C:\conda\Anaconda3\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
C:\conda\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\conda\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
